# Ejercicio 3. Subtitulado de imágenes con Keras

Este ejercicio está basado en esta web: [Image captioning with Keras](https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8), la cual a su vez está basado en el trabajo de [Karphaty et al.](https://cs.stanford.edu/people/karpathy/deepimagesent/) en el subtitulado automático de imágenes (image captioning, en inglés). Se pretende crear un modelo que dada una imagen, devuelva un texto describiendo lo que ocurre en ella. Para ello vamos a tener que usar técnicas como CNN, transfer learning, word embedding, recurrent neural networks y modelos no secuenciales.

![image captioning](img/imagecaptioning.jpg)

## 1. Enunciado

Basándote en el [trabajo original](https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8) (cuyo notebook con todo el código se puede descargar en este [repositorio github](https://github.com/hlamba28/Automatic-Image-Captioning)), se pide hacer **una variante** del trabajo original con las siguientes modificaciones:
* Usando un modelo pre-entrenado distinto tanto para la extracción de características (es decir, que no sea InceptionV3) como para el word embedding (que no sea Glove.6B con 200d; es decir, puede ser otro Glove, Word2Vec, ...).
* Cambiando los hiperparámetros empleados y la arquitectura del modelo final: usar otro factor de aprendizaje, usar otro método de optimización (SGD, Adam, RMSprop, ...), aplicar regularización, en la parte recurrente, usar otro tipo de red que no sea LSTM (como la GRU), añadiendo capas en el modelo (al menos una, ¿donde la insertarías?, etc.

Después, re-entrena el modelo, evalúalo y analiza los resultados obtenidos, comparándolo con los del trabajo original. Debes razonar los cambios elegidos. 

*De forma opcional*, se valorará la construcción de más variantes, el empleo de modelos pre-entrenados (CNN o Embedding) distintos a los vistos en la asignatura y el uso de otro dataset que no sea Flickr8k.

## 2. Entrega

La entrega de este ejercicio se realiza a través de la tarea creada para tal efecto en Enseñanza Virtual. Tienes que entregar un notebook, y el HTML generado a partir de él, cuyas celdas estén ya evaluadas.

La estructura del notebook debe contener los siguientes apartados:

0. Cabecera: nombre y apellidos.
1. Dataset: descripción, carga y visualización.
2. Preparación de los datos para ser usados en Keras.
3. Modelos y configuraciones creadas en Keras y su entrenamiento. Debe incluir una explicación razonada de los cambios realizados con tus palabras.
5. Análisis de resultados con comparativa respecto del trabajo original (*no es necesario mejorarlo*). Debe incluir pruebas con imágenes del conjunto de test.
6. Bibliografía utilizada (enlaces web, material de clase, libros, etc.).

### 2.1. Nota importante
-----
**HONESTIDAD ACADÉMICA Y COPIAS: un trabajo práctico es un examen, por lo que
debe realizarse de manera individual. La discusión y el intercambio de
información de carácter general con los compañeros se permite (e incluso se
recomienda), pero NO AL NIVEL DE CÓDIGO. Igualmente el remitir código de
terceros, OBTENIDO A TRAVÉS DE LA RED o cualquier otro medio, se considerará
plagio.** 

**Cualquier plagio o compartición de código que se detecte significará
automáticamente la calificación de CERO EN LA ASIGNATURA para TODOS los
alumnos involucrados. Por tanto a estos alumnos NO se les conservará, para
futuras convocatorias, ninguna nota que hubiesen obtenido hasta el momento.
SIN PERJUICIO DE OTRAS MEDIDAS DE CARÁCTER DISCIPLINARIO QUE SE PUDIERAN
TOMAR.**

-----

## 3. El dataset

Hay muchos conjuntos de datos de código abierto disponibles para este problema, como **Flickr 8k** (que contiene 8.000 imágenes), Flickr 30k (que contiene 30.000 imágenes), MS COCO (que contiene 180.000 imágenes), etc. Pero para el propósito de este ejercicio, vamos a utilizar el conjunto de datos de Flickr 8k. 

En teoría se puede descargar rellenando [este formulario](https://forms.illinois.edu/sec/1713398) proporcionado por la Universidad de Illinois en Urbana-Champaign. Si el link proporcionado después de rellenar el formulario falla (sucede a fecha de octubre de 2019), puedes usar los siguientes enlaces para descargarlo de manera directa:
* [Flickr8k_Dataset](https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip)
* [Flickr8k_Text](https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip)

Créditos: Jason Brownlee

## 4. El modelo

La idea de esta solución se basa en la construcción de un modelo no secuencial en Keras, donde tenemos dos entradas:
* La imagen
* Un caption parcial (es decir, una frase sin completar)

Y la salida obtenida es la siguiente palabra.

![modelo](img/modeloic.jpg)

Yendo más al detalle, tenemos que:
* La **entrada 1** correspondiente a la imagen es en realidad el conjunto de características extraídas de un modelo pre-entrenado.
* La **entrada 2** correspondiente a la secuencia de palabras que forman un caption parcial (una frase sin completar) siempre empieza y acaba por 'startseq' y termina en 'endseq'. 
* La **salida** es una capa softmax donde eligiremos la palabra con más probabilidad como la siguiente.

En el modelo por tanto tenemos las siguientes ramas:
* Entrada 1, que es un tensor de 2048 elementos (valor que podría variar con otro modelo pre-entrenado!), y pasa por una capa de dropout y por una capa densa.
* Entrada 2, que es un tensor de 34 elementos (la longitud del mayor caption), y que se hace pasar por una capa Embedding (con pesos pre-entrenados de Glove.6B) de dimensión 200. Después, esto se hace pasar por una capa dropout, y como se trata de una secuencia, la hacemos pasar también por una red recurrente, en este caso, una LSTM.

Estas dos ramas se unen con la función `add` de Keras, y después se pasa por una capa densa y la capa softmax.

Para hacer inferencia sobre una imagen (suponiendo que el modelo ya esté entrenado), el proceso sería el siguiente:
* Primero se insertan las características extraídas de la imagen por entrada 1, y la secuencia que solo contiene "startseq" por entrada 2, y se infiere una palabra.
* Segudo se insertan de nuevo las características extraídas de la imagen por entrada 1, y la secuencia se actualiza concatenando lo anterior con la palabra inferida anteriormente. Si se ha inferido "the", entonces "startseq the". Se infiere otra palabra.
* Repetir el proceso hasta conseguir "endseq" o el límite del tamaño del caption (34).

Para entrenar el modelo, donde sabems qué captions se pueden dar para una imagen (habrá más de una posibilidad), lo que se hace es generar distintos ejemplos para una misma imagen y caption. Verás que hay definido una función data_generator, el cual devuelve un generador de python que será usado por Keras (por eso se usa la palabra reservada `yield` y no `result`). Aquí se crea un datapoint por cada imagen `a`, después por cada caption `c`, y después por cada palabra `i` en el caption, creando (x1,x2,y1), siendo x1 las características de la imagen `a`, x2 la subsecuencia de las palabras antes de `c` antes de la palabra `i`, y y1 la palabra `i`. Visualmente, hay que generar todos estos datos para las primeras dos imágenes:

![datosic](img/dataic.jpg)

Todos estos métodos no tienes que cambiarlos, a no ser que quieras mejorarlos. Úsalos directamente, en este ejercicio se pide jugar con el modelo construido: cambiando las capas, cambiando el modelo pre-entrenado para extraer características, cambiando el embedding, etc. Recuerda que tienes todos los detalles en los enlaces que se han dado arriba.